In [1]:
from numpy import mean
from sklearn.datasets import make_classification
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RepeatedStratifiedKFold
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

dataset=pd.read_csv('https://raw.githubusercontent.com/mahayasa/various-sampling-churn-prediction/main/data/telco-non-internet.csv')
dataset=dataset.dropna()
#dataset=dataset.fillna(0)

dataset['state']=le.fit_transform(dataset['state'])
dataset['area_code']=le.fit_transform(dataset['area_code'])
dataset['international_plan']=le.fit_transform(dataset['international_plan'])
dataset['voice_mail_plan']=le.fit_transform(dataset['voice_mail_plan'])
dataset['churn']=le.fit_transform(dataset['churn'])

#filling missing value with mean
#dataset['TotalCharges'].replace(to_replace = 0, value = dataset['TotalCharges'].mean(), inplace=True)
X=dataset.drop(['churn'],axis=1)
y=dataset["churn"]

In [2]:
# decision tree
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier()
# define grid
balance = [{0:0,1:0},{0:100,1:1}, {0:10,1:1}, {0:1,1:1}, {0:1,1:10}, {0:1,1:100}]
param_grid = dict(class_weight=balance)
# define evaluation procedure
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)
# define grid search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=cv, scoring='roc_auc')
# execute the grid search
grid_result = grid.fit(X, y)
# report the best configuration
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# report all configurations
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.834941 using {'class_weight': {0: 1, 1: 1}}
0.500000 (0.000000) with: {'class_weight': {0: 0, 1: 0}}
0.831033 (0.025528) with: {'class_weight': {0: 100, 1: 1}}
0.834516 (0.022175) with: {'class_weight': {0: 10, 1: 1}}
0.834941 (0.023626) with: {'class_weight': {0: 1, 1: 1}}
0.831340 (0.022149) with: {'class_weight': {0: 1, 1: 10}}
0.813891 (0.018889) with: {'class_weight': {0: 1, 1: 100}}


In [3]:
from sklearn.metrics import f1_score, make_scorer
f1 = make_scorer(f1_score , average='macro')
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=cv, scoring=f1)
# execute the grid search
grid_result = grid.fit(X, y)
# report the best configuration
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# report all configurations
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.832057 using {'class_weight': {0: 1, 1: 10}}
0.462161 (0.000167) with: {'class_weight': {0: 0, 1: 0}}
0.818170 (0.019165) with: {'class_weight': {0: 100, 1: 1}}
0.822679 (0.020208) with: {'class_weight': {0: 10, 1: 1}}
0.829460 (0.016471) with: {'class_weight': {0: 1, 1: 1}}
0.832057 (0.017142) with: {'class_weight': {0: 1, 1: 10}}
0.818287 (0.014612) with: {'class_weight': {0: 1, 1: 100}}


In [4]:
from imblearn.metrics import geometric_mean_score
gm_scorer = make_scorer(geometric_mean_score, greater_is_better=True, average='binary')

grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=cv, scoring=gm_scorer)
# execute the grid search
grid_result = grid.fit(X, y)
# report the best configuration
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# report all configurations
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.829656 using {'class_weight': {0: 1, 1: 1}}
0.000000 (0.000000) with: {'class_weight': {0: 0, 1: 0}}
0.824921 (0.028522) with: {'class_weight': {0: 100, 1: 1}}
0.826698 (0.026033) with: {'class_weight': {0: 10, 1: 1}}
0.829656 (0.026384) with: {'class_weight': {0: 1, 1: 1}}
0.823653 (0.022287) with: {'class_weight': {0: 1, 1: 10}}
0.797788 (0.021041) with: {'class_weight': {0: 1, 1: 100}}


In [5]:
# RF
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier()
# define grid
balance = [{0:0,1:0},{0:100,1:1}, {0:10,1:1}, {0:1,1:1}, {0:1,1:10}, {0:1,1:100}]
param_grid = dict(class_weight=balance)
# define evaluation procedure
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)
# define grid search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=cv, scoring='roc_auc')
# execute the grid search
grid_result = grid.fit(X, y)
# report the best configuration
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# report all configurations
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.917921 using {'class_weight': {0: 1, 1: 10}}
0.500000 (0.000000) with: {'class_weight': {0: 0, 1: 0}}
0.916103 (0.020915) with: {'class_weight': {0: 100, 1: 1}}
0.916241 (0.022253) with: {'class_weight': {0: 10, 1: 1}}
0.917011 (0.022323) with: {'class_weight': {0: 1, 1: 1}}
0.917921 (0.019185) with: {'class_weight': {0: 1, 1: 10}}
0.914000 (0.016971) with: {'class_weight': {0: 1, 1: 100}}


In [6]:
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=cv, scoring=f1)
# execute the grid search
grid_result = grid.fit(X, y)
# report the best configuration
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# report all configurations
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.901386 using {'class_weight': {0: 1, 1: 1}}
0.462161 (0.000167) with: {'class_weight': {0: 0, 1: 0}}
0.892810 (0.013376) with: {'class_weight': {0: 100, 1: 1}}
0.898034 (0.013807) with: {'class_weight': {0: 10, 1: 1}}
0.901386 (0.014143) with: {'class_weight': {0: 1, 1: 1}}
0.883018 (0.014930) with: {'class_weight': {0: 1, 1: 10}}
0.854210 (0.022144) with: {'class_weight': {0: 1, 1: 100}}


In [7]:
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=cv, scoring=gm_scorer)
# execute the grid search
grid_result = grid.fit(X, y)
# report the best configuration
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# report all configurations
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.859863 using {'class_weight': {0: 10, 1: 1}}
0.000000 (0.000000) with: {'class_weight': {0: 0, 1: 0}}
0.851325 (0.024937) with: {'class_weight': {0: 100, 1: 1}}
0.859863 (0.024606) with: {'class_weight': {0: 10, 1: 1}}
0.855056 (0.019023) with: {'class_weight': {0: 1, 1: 1}}
0.825486 (0.023019) with: {'class_weight': {0: 1, 1: 10}}
0.788941 (0.030034) with: {'class_weight': {0: 1, 1: 100}}


In [8]:
# LGB
from lightgbm import LGBMClassifier
model = LGBMClassifier()
# define grid
balance = [{0:0,1:0},{0:100,1:1}, {0:10,1:1}, {0:1,1:1}, {0:1,1:10}, {0:1,1:100}]
param_grid = dict(class_weight=balance)
# define evaluation procedure
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)
# define grid search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=cv, scoring='roc_auc')
# execute the grid search
grid_result = grid.fit(X, y)
# report the best configuration
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# report all configurations
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

[LightGBM] [Info] Number of positive: 598, number of negative: 3652
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000683 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2491
[LightGBM] [Info] Number of data points in the train set: 4250, number of used features: 19
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.620847 -> initscore=0.493146
[LightGBM] [Info] Start training from score 0.493146
Best: 0.920203 using {'class_weight': {0: 1, 1: 10}}
0.500000 (0.000000) with: {'class_weight': {0: 0, 1: 0}}
0.910918 (0.020370) with: {'class_weight': {0: 100, 1: 1}}
0.919867 (0.015927) with: {'class_weight': {0: 10, 1: 1}}
0.918587 (0.014060) with: {'class_weight': {0: 1, 1: 1}}
0.920203 (0.014506) with: {'class_weight': {0: 1, 1: 10}}
0.919069 (0.015332) with: {'class_weight': {0: 1, 1: 100}}


In [9]:
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=cv, scoring=f1)
# execute the grid search
grid_result = grid.fit(X, y)
# report the best configuration
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# report all configurations
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

[LightGBM] [Info] Number of positive: 598, number of negative: 3652
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000705 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2491
[LightGBM] [Info] Number of data points in the train set: 4250, number of used features: 19
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.140706 -> initscore=-1.809439
[LightGBM] [Info] Start training from score -1.809439
Best: 0.910140 using {'class_weight': {0: 1, 1: 1}}
0.462161 (0.000167) with: {'class_weight': {0: 0, 1: 0}}
0.865183 (0.018350) with: {'class_weight': {0: 100, 1: 1}}
0.895184 (0.013212) with: {'class_weight': {0: 10, 1: 1}}
0.910140 (0.009861) with: {'class_weight': {0: 1, 1: 1}}
0.908626 (0.011567) with: {'class_weight': {0: 1, 1: 10}}
0.877243 (0.013951) with: {'class_weight': {0: 1, 1: 100}}


In [10]:
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=cv, scoring=gm_scorer)
# execute the grid search
grid_result = grid.fit(X, y)
# report the best configuration
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# report all configurations
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

[LightGBM] [Info] Number of positive: 598, number of negative: 3652
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000571 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2491
[LightGBM] [Info] Number of data points in the train set: 4250, number of used features: 19
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.620847 -> initscore=0.493146
[LightGBM] [Info] Start training from score 0.493146
Best: 0.896777 using {'class_weight': {0: 1, 1: 10}}
0.000000 (0.000000) with: {'class_weight': {0: 0, 1: 0}}
0.794327 (0.027529) with: {'class_weight': {0: 100, 1: 1}}
0.837601 (0.021457) with: {'class_weight': {0: 10, 1: 1}}
0.873178 (0.017483) with: {'class_weight': {0: 1, 1: 1}}
0.896777 (0.018043) with: {'class_weight': {0: 1, 1: 10}}
0.894234 (0.018740) with: {'class_weight': {0: 1, 1: 100}}


In [11]:
#XGB
from xgboost import XGBClassifier
model = XGBClassifier()
# define grid
weights = [1, 10, 25, 50, 75, 99, 100, 1000]
param_grid = dict(scale_pos_weight=weights)
# define evaluation procedure
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)
# define grid search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=cv, scoring='roc_auc')
# execute the grid search
grid_result = grid.fit(X, y)
# report the best configuration
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# report all configurations
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.919044 using {'scale_pos_weight': 25}
0.915732 (0.019307) with: {'scale_pos_weight': 1}
0.918982 (0.016085) with: {'scale_pos_weight': 10}
0.919044 (0.017073) with: {'scale_pos_weight': 25}
0.918042 (0.018198) with: {'scale_pos_weight': 50}
0.917560 (0.014424) with: {'scale_pos_weight': 75}
0.916533 (0.015205) with: {'scale_pos_weight': 99}
0.915970 (0.017779) with: {'scale_pos_weight': 100}
0.915338 (0.017271) with: {'scale_pos_weight': 1000}


In [12]:
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=cv, scoring=f1)
# execute the grid search
grid_result = grid.fit(X, y)
# report the best configuration
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# report all configurations
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.908329 using {'scale_pos_weight': 25}
0.905658 (0.013522) with: {'scale_pos_weight': 1}
0.905182 (0.011267) with: {'scale_pos_weight': 10}
0.908329 (0.012361) with: {'scale_pos_weight': 25}
0.903275 (0.011570) with: {'scale_pos_weight': 50}
0.899118 (0.010410) with: {'scale_pos_weight': 75}
0.896146 (0.014574) with: {'scale_pos_weight': 99}
0.896951 (0.010578) with: {'scale_pos_weight': 100}
0.871209 (0.012442) with: {'scale_pos_weight': 1000}


In [13]:
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=cv, scoring=gm_scorer)
# execute the grid search
grid_result = grid.fit(X, y)
# report the best configuration
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# report all configurations
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.893715 using {'scale_pos_weight': 25}
0.869632 (0.018649) with: {'scale_pos_weight': 1}
0.886007 (0.018833) with: {'scale_pos_weight': 10}
0.893715 (0.018751) with: {'scale_pos_weight': 25}
0.891550 (0.015097) with: {'scale_pos_weight': 50}
0.891587 (0.014515) with: {'scale_pos_weight': 75}
0.890149 (0.020029) with: {'scale_pos_weight': 99}
0.892387 (0.017457) with: {'scale_pos_weight': 100}
0.881340 (0.017194) with: {'scale_pos_weight': 1000}
